In [72]:
import pandas as pd

# Load the dataset
tv_df = pd.read_csv("input_data/estat_isoc_ci_dev_i_en.csv")
beds_df = pd.read_csv("input_data/estat_tour_cap_nat_en.csv")

In [58]:
print(beds_df.columns)
print(tv_df.columns)

Index(['DATAFLOW', 'LAST UPDATE', 'freq', 'accomunit', 'unit', 'nace_r2',
       'geo', 'TIME_PERIOD', 'OBS_VALUE', 'OBS_FLAG', 'CONF_STATUS'],
      dtype='object')
Index(['DATAFLOW', 'LAST UPDATE', 'freq', 'ind_type', 'indic_is', 'unit',
       'geo', 'TIME_PERIOD', 'OBS_VALUE', 'OBS_FLAG', 'CONF_STATUS'],
      dtype='object')


In [59]:
# cols to keep
beds_df = beds_df[['geo','TIME_PERIOD','accomunit', 'unit', 'nace_r2','OBS_VALUE', 'OBS_FLAG']]
print(beds_df.head())

  geo  TIME_PERIOD accomunit unit nace_r2  OBS_VALUE OBS_FLAG
0  AT         1990     BEDPL   NR    I551   650559.0      NaN
1  AT         1991     BEDPL   NR    I551   654127.0      NaN
2  AT         1992     BEDPL   NR    I551   653586.0      NaN
3  AT         1993     BEDPL   NR    I551   651003.0      NaN
4  AT         1994     BEDPL   NR    I551   650020.0      NaN


In [63]:
# print(beds_df['geo'].value_counts())
# print(beds_df['nace_r2'].value_counts())
# print(beds_df['accomunit'].value_counts())
# print(beds_df['unit'].value_counts())
# print(beds_df['TIME_PERIOD'].value_counts())
print(beds_df['OBS_FLAG'].value_counts())

OBS_FLAG
e     705
b     447
u     125
be      6
Name: count, dtype: int64


In [61]:
beds_df.dtypes

geo             object
TIME_PERIOD      int64
accomunit       object
unit            object
nace_r2         object
OBS_VALUE      float64
OBS_FLAG        object
dtype: object

In [62]:
# Rows to use: You should only use rows where: 
# “accomunit” is “BEDPL”, --> accomodation unit : bedplaces
# “unit” is “NR”, --> number
# “nace_r2” is “I551”, --> accomodation type : hotels and similar accommodation 
# “TIME_PERDIO” is “2016” 

# Output the number of bedplaces in hotels and similar accommodation in 2016 for each country
beds_df_2016 = beds_df[(beds_df['accomunit'] == 'BEDPL') & (beds_df['unit'] == 'NR') & (beds_df['nace_r2'] == 'I551') & (beds_df['TIME_PERIOD'] == 2016)]
print(beds_df_2016.head())

    geo  TIME_PERIOD accomunit unit nace_r2  OBS_VALUE OBS_FLAG
26   AT         2016     BEDPL   NR    I551   601945.0      NaN
60   BE         2016     BEDPL   NR    I551   129232.0      NaN
94   BG         2016     BEDPL   NR    I551   281869.0      NaN
126  CH         2016     BEDPL   NR    I551   271710.0      NaN
159  CY         2016     BEDPL   NR    I551    83251.0      NaN


In [ ]:
#clean up the data

import numpy as np
# Remove rows where geo is EA, EU27_2007, EU27_2020, EU28
beds_df_2016 = beds_df_2016[~beds_df_2016['geo'].str.contains('EA|EU')]

# Remove rows where OBS_VALUE is not a number, Replace ':' with NaN and convert to float
beds_df_2016['OBS_VALUE'] = beds_df_2016['OBS_VALUE'].replace(':', np.nan).astype(float)
beds_df_2016 = beds_df_2016.dropna(subset=['OBS_VALUE'])

# Remove rows where OBS_FLAG contains 'u' or 'bu' as they are not reliable
beds_df_2016 = beds_df_2016[~beds_df_2016['geo'].str.contains('u|bu')]

In [55]:
#check the data
print(tv_df.head())

                   DATAFLOW        LAST UPDATE freq   ind_type    indic_is  \
0  ESTAT:ISOC_CI_DEV_I(1.0)  17/12/24 11:00:00    A  CB_EU_FOR  I_IUG_DKPC   
1  ESTAT:ISOC_CI_DEV_I(1.0)  17/12/24 11:00:00    A  CB_EU_FOR  I_IUG_DKPC   
2  ESTAT:ISOC_CI_DEV_I(1.0)  17/12/24 11:00:00    A  CB_EU_FOR  I_IUG_DKPC   
3  ESTAT:ISOC_CI_DEV_I(1.0)  17/12/24 11:00:00    A  CB_EU_FOR  I_IUG_DKPC   
4  ESTAT:ISOC_CI_DEV_I(1.0)  17/12/24 11:00:00    A  CB_EU_FOR  I_IUG_DKPC   

     unit geo  TIME_PERIOD  OBS_VALUE OBS_FLAG  CONF_STATUS  
0  PC_IND  AL         2018        NaN        u          NaN  
1  PC_IND  AT         2016      37.41      NaN          NaN  
2  PC_IND  AT         2018      42.45      NaN          NaN  
3  PC_IND  AT         2021      39.62      NaN          NaN  
4  PC_IND  AT         2023      40.34      NaN          NaN  


In [66]:
tv_df.columns

Index(['DATAFLOW', 'LAST UPDATE', 'freq', 'ind_type', 'indic_is', 'unit',
       'geo', 'TIME_PERIOD', 'OBS_VALUE', 'OBS_FLAG', 'CONF_STATUS'],
      dtype='object')

In [73]:
# cols to keep
tv_df = tv_df[['geo','ind_type','indic_is', 'unit', 'TIME_PERIOD','OBS_VALUE', 'OBS_FLAG']]

In [85]:
# rows to use: You should only use rows where:
# · “ind_type” is “IND_TOTAL” --> individual type such as students, males etc
# · “indic_is” is “I_IUG_TV” --> “indic_is”: individuals who used internet on type of device such as TV, computer, mobile --> Selecting only individuals who used internet on smart TV
# · “unit” is “PC_IND”
# · “TIME_PERIOD” is “2016”

# Output the number of TV sets per 100 inhabitants in 2016 for each country
tv_df_2016 = tv_df[(tv_df['ind_type'] == 'IND_TOTAL') & (tv_df['indic_is'] == 'I_IUG_TV') & (tv_df['unit'] == 'PC_IND') & (tv_df['TIME_PERIOD'] == 2016)]
print(tv_df_2016.head())


      geo   ind_type  indic_is    unit  TIME_PERIOD  OBS_VALUE OBS_FLAG
97734  AT  IND_TOTAL  I_IUG_TV  PC_IND         2016      14.78      NaN
97735  BE  IND_TOTAL  I_IUG_TV  PC_IND         2016       9.89      NaN
97736  BG  IND_TOTAL  I_IUG_TV  PC_IND         2016       3.71      NaN
97737  CY  IND_TOTAL  I_IUG_TV  PC_IND         2016       6.41      NaN
97738  CZ  IND_TOTAL  I_IUG_TV  PC_IND         2016       5.32      NaN


In [75]:
tv_df_2016['OBS_FLAG'].value_counts()

OBS_FLAG
b    2
Name: count, dtype: int64

In [ ]:
# CLean up the data
import numpy as np
# Special values
# ● The value “: ” signifies missing data.
# ● Rows where OBS_FLAG contains “u” or “bu” are unreliable and should be considered as missing data.
# ● Country codes “EA”, "EU27_2007", "EU27_2020";, "EU28"; should be treated the same as missing data and should be ignored.[2]

#Treat the special values as missing data
tv_df_2016 = tv_df_2016[~tv_df_2016['geo'].str.contains('EA|EU')]

In [81]:
# Replace ":" with NaN in 'OBS_VALUE' and then drop rows with NaN
# tv_df_2016['OBS_VALUE'] = tv_df_2016['OBS_VALUE'].replace(':', np.nan).astype(float)
# tv_df_2016 = tv_df_2016.dropna(subset=['OBS_VALUE'])

# Remove rows where OBS_FLAG contains 'u' or 'bu' as they are not reliable: There is "b" and i am assuming it is a typo and should be "bu", hence removing both
tv_df_2016 = tv_df_2016[~tv_df_2016['OBS_FLAG'].isin(['u', 'bu', 'b'])]

In [99]:
#left merge the two dataframes on 'geo'
geo_merged = pd.merge(beds_df_2016, tv_df_2016, on='geo', how='left', suffixes=('_beds', '_device'))
# keep only the columns we need: “Country Code”, “Percentage of individuals” and “Number of Bed-places”.
geo_merged = geo_merged[['geo', 'OBS_VALUE_beds', 'OBS_VALUE_device']]
# Rename the columns to “Country Code”, “Number of Bed-places” and “Percentage of individuals”
geo_merged.columns = ['Country Code', 'Number of Bed-places', 'Percentage of individuals']

# Find the 'geo' values where 'Percentage of individuals' is missing
geo_missing_tv = geo_merged[geo_merged['Percentage of individuals'].isnull()]['Country Code'].tolist()

# Print the list of missing 'geo' values
print(f"Missing geo values in tv_df_2016: {geo_missing_tv}")

# Create a dataframe containing the missing 'geo' values and the 'Percentage of individuals' from tv_df_2016
missing_tv_df = tv_df_2016[tv_df_2016['geo'].isin(geo_missing_tv)][['geo', 'OBS_VALUE']].rename(columns={'OBS_VALUE': 'Percentage of individuals'})
print(missing_tv_df)

Missing geo values in tv_df_2016: ['CH', 'IS', 'LI', 'ME', 'TR']
Empty DataFrame
Columns: [geo, Percentage of individuals]
Index: []


In [97]:
# For simplicity i will remove rows where tv related data is not available. However, a better solution would be to fill the missing values with previous year data or some other method.
geo_merged = geo_merged.dropna(subset=['Percentage of individuals'])

In [ ]:

import os

# Create the output directory if it doesn't exist
if not os.path.exists('output_data'):
	os.makedirs('output_data')

# Save the merged dataframe to a CSV file
geo_merged.to_csv('output_data/geo_beds_tv_df.csv', index=False, mode='w')
print(geo_merged.head())

  Country Code  Number of Bed-places  Percentage of individuals
0           AT              601945.0                      14.78
1           BE              129232.0                       9.89
2           BG              281869.0                       3.71
3           CH              271710.0                        NaN
4           CY               83251.0                       6.41


<!-- # Visualize the data for Outcome
# Can I Multiply "Number of Bed-places" by "% of Individuals" to Estimate Target Users?
# I can. BUT! This is a rough estimate and has some assumptions and limitations.

# What the Percentage Means:
- The "% of individuals" is the share of the general population who use a TV to access the internet, not specifically hotel guests. This assumes hotel guests are similar to the general population in their tech habits.
- Bed-places ≠ Unique Guests: "Bed-places" is the number of beds available, not the number of guests per year. Actual guest numbers could be higher (if beds are used by multiple guests per year) or lower (if occupancy is low).
- Not All Guests Will Use the Feature: Even if someone uses a smart TV at home, they may not use it in a hotel.
- Market Size Estimate: This calculation gives you a relative market size for comparison between countries. -->